<a href="https://colab.research.google.com/github/shahzadsiddiqi/BERT-LLM-tasks/blob/main/BERT_for_a_text_classification_task_(Sentiment_analysis).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup in Colab
First, make sure the transformers library is installed.

In [15]:
!pip install transformers datasets torch


Step 1: Import Libraries

In [16]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torch


Step 2: Load and Prepare Dataset
We'll use the IMDB dataset for sentiment classification.

In [17]:
# Load IMDB dataset
dataset = load_dataset("imdb")

# Split into train/test datasets
# Use only a subset of the training data
train_dataset = dataset["train"].shuffle(seed=42).select(range(400))  # Smaller dataset (e.g., 400 examples)
test_dataset = dataset["test"].shuffle(seed=42).select(range(100))    # Smaller test set



Step 3: Tokenization
Preprocess text inputs by tokenizing with the BERT tokenizer.

In [18]:
# Load pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for PyTorch tensors
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


Step 4: Load BERT Model
Load the pre-trained BERT model with a classification head.

In [19]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)  # Binary classification (Positive/Negative)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step 5: Define Training Arguments and Trainer
Set up the Hugging Face Trainer for training.

In [20]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=10,
    push_to_hub=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step 6: Train the Model

In [21]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.567900,0.566684
2,0.341100,0.426503


TrainOutput(global_step=100, training_loss=0.5296570181846618, metrics={'train_runtime': 54.9756, 'train_samples_per_second': 14.552, 'train_steps_per_second': 1.819, 'total_flos': 52622211072000.0, 'train_loss': 0.5296570181846618, 'epoch': 2.0})

Step 7: Evaluate the Model

In [22]:
# Evaluate on the test dataset
results = trainer.evaluate()

print("Evaluation Results:", results)


Evaluation Results: {'eval_loss': 0.4265032708644867, 'eval_runtime': 0.6886, 'eval_samples_per_second': 145.222, 'eval_steps_per_second': 18.879, 'epoch': 2.0}


Step 8: Make Predictions
You can make predictions on raw text using the fine-tuned model.

In [24]:
import torch

# Function to predict sentiment
def predict_sentiment(text):
    # Move the model to the correct device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Ensure the model is on GPU or CPU as applicable

    # Tokenize and move inputs to the same device
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to device

    # Run through the model
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1).item()

    # Map prediction to label
    return "Positive" if predictions == 1 else "Negative"

# Test the model on new examples
print(predict_sentiment("The movie was fantastic and engaging!"))  # Expected: Positive
print(predict_sentiment("I hated the film. It was so boring."))    # Expected: Negative


Positive
Negative


Expected Outputs
During training, you will see metrics like loss and accuracy improving after every epoch.
Evaluation results will give metrics such as eval_loss and eval_accuracy.
The predictions on the test examples should correctly classify the sentiment as positive or negative.